Author: Samuel Hori, Blue Dragon  
V0.0
Are you tired of clicking to queue matches? Do you need a complete set of matches for you (as of yet, unwritten) match analyzer). Here's an auto-scrim submitter. Have fun.

This entire program is rather trivial and very easy to modify if you want but I've included some basic functionality to let you queue groups of games you might be interested in. Let me know if there are features you think should be 

In [ ]:
import pickle
import re
import requests
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
auth_token=#Put Your AuthCode Here (from https://api.battlecode.org/api/token/)
my_team=#Put your team here

In [ ]:
teams=dict()
elos=dict()

for i in range(1,1000):
    r = requests.get('https://api.battlecode.org/api/team/bc23/t/',params={"page":i})
    if(r.status_code==200):
        for t in r.json()["results"]:
            teams[t["name"]]=t["id"]
            elos[t["id"]]=t["profile"]["rating"]

In [ ]:
print(teams)

In [ ]:
def requestMatch(team, map_names,ranked=False, playerOrder="?"):
    #To change player order use "+" to make yourself team 1, "-" to be team 2 and "?" to alternate
    params={
      "is_ranked": ranked,
      "requested_to": team,
      "player_order": playerOrder,
      "map_names": map_names
    }
    r = requests.post('https://api.battlecode.org/api/compete/bc23/request/',json=params,headers={"Authorization": "Bearer "+auth_token})
    if(r.status_code!=201):
        print(r.status_code)
        print(r.text)

In [ ]:
maps= requests.get('https://api.battlecode.org/api/episode/bc23/map/').json()
print(maps)
map_names=[]
for m in maps:
    if(m["is_public"]):
        map_names.append(m["name"])

In [ ]:
#Request all maps against a single team
def requestAllMaps(team,ranked=False,playerOrder="?"):
    mapsToUse=set(map_names)
    while(len(mapsToUse)>0):
        if(len(mapsToUse)>=3):
            maps=set(random.sample(mapsToUse, 3))
            requestMatch(team,list(maps),ranked=ranked,playerOrder=playerOrder)
            mapsToUse-=maps
        else:
            maps=set(random.sample(mapsToUse, len(mapsToUse)))
            maps=maps.union(set(random.sample(set(map_names)-maps, 3-len(maps))))
            mapsToUse-=maps
            requestMatch(team,list(maps),ranked=ranked,playerOrder=playerOrder)
def requestAllMapsByOnes(team,ranked=False,playerOrder="?"):
    mapsToUse=map_names
    for m in mapsToUse:
        requestMatch(team,[m],ranked=ranked,playerOrder=playerOrder)
            

In [ ]:
#Get teams within N Elo
def getComparableTeams(team="Blue Dragon",upperRange=20,lowerRange=20):
    team_id=teams[team]
    elo=elos[team_id]
    comparableTeams=[]
    for e in elos.keys():
        if(elos[e]<elo+upperRange and elos[e]>elo-lowerRange):
            comparableTeams.append(e)
    return comparableTeams

A few examples of how to run a bunch of matches at once

In [ ]:
#Queue games against similar elo teams
teamsToFight=getComparableTeams(team=my_team,upperRange=100,lowerRange=100)
print("Number of Teams Queued: ",len(teamsToFight))
for team in teamsToFight:
    print({v:k for v,k in zip(teams.values(),teams.keys())}[team])
    #Randomly choose a single map to play against each
    m=random.sample(map_names, 1)
    requestMatch(team,map_names=m,ranked=False,playerOrder="?")

In [ ]:
#If you are interested in a specific team---can be done like this
teamToFight="Teh Devs"   
#queue a random map
m=random.sample(map_names, 1)
requestMatch(teams[teamToFight],m,ranked=False,playerOrder="?")

In [ ]:
#run every map against them 1 by 1 (useful for my future plans)
#requestAllMapsByOnes(teams[teamToFight],ranked=False,playerOrder="?")

In [ ]:
#request all in groups of 3 (default number)
#requestAllMaps(teams[teamToFight],ranked=False,playerOrder="?")